In [1]:
import cv2
import tqdm
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf


/Users/ataberkuygar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:


dataset = {
    "dataset_path": os.path.join("dataset", "Fit3D Video Dataset"),
    "sequence_length": 75,
    "resize_width": 128,
    "resize_height": 128,
    "color_channels": 1,
    "validation_ratio": 0.2,
    "test_ratio": 0.2
}

In [3]:
def get_mean_std_frame(frames):
       mean = tf.math.reduce_mean(frames)
       std = tf.math.reduce_std(tf.cast(frames, tf.float32))
       return tf.cast((frames - mean), tf.float32) / std

In [4]:
def process_frame(frame):
       resized_frame = tf.image.resize(frame, [dataset["resize_height"], dataset["resize_width"]])
       gray_scaled_image = tf.image.rgb_to_grayscale(resized_frame)
       gray_scaled_image = tf.cast(gray_scaled_image, tf.float32)
       normalized_frame = gray_scaled_image / 255.0
       
       return normalized_frame

In [5]:

def read_video(video_path):
    video_reader = cv2.VideoCapture(video_path)
    frames = []
    if not video_reader.isOpened():
        print("Error: Could not open video.")
        return
    
    frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(frame_count / dataset["sequence_length"]), 1)
    
    with tqdm.tqdm(bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_noinv_fmt}]", 
                    desc="Reading sequence frames...", unit=" frames", total=dataset["sequence_length"], leave=False) as pbar_frames:
        for frame_index in range(dataset["sequence_length"]):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, (frame_index * skip_frames_window))
            
            success, frame = video_reader.read()
        
            if frame is None:
                break
        
            if not success:
                print("Error: Could not read frame.")
                break
            
            processed_frame = process_frame(frame)
            frames.append(processed_frame)
            pbar_frames.update(1)
            
    video_reader.release()
    return get_mean_std_frame(frames)

In [6]:
PATH="dataset/Fit3D Video Dataset"
tagged_paths = {}
categories = os.listdir(PATH)
categories 

['Walk The Box',
 '.DS_Store',
 'Mule Kick',
 'Squat',
 'Band Pull Apart',
 'Man Maker',
 'Pushup',
 'Burpees']

In [7]:
tagged_paths = {}
categories = os.listdir(dataset["dataset_path"])
for category in categories:
    if category == '.DS_Store':
        continue  # Skip this item
    if category not in tagged_paths:
        tagged_paths[category] = []
    print(f"Processing category: {category}")    
    folder_path = os.path.join(dataset["dataset_path"], category)
    video_list = os.listdir(folder_path)
    for video in video_list:
        if video == '.DS_Store':
            continue  # Skip this item
        video_path = os.path.join(folder_path, video)
        tagged_paths[category].append(tf.convert_to_tensor(video_path))
        print(video_list)

Processing category: Walk The Box
['walk_the_box_11.mp4', 'walk_the_box_10.mp4', 'walk_the_box_12.mp4', 'walk_the_box_13.mp4', 'walk_the_box_17.mp4', 'walk_the_box_16.mp4', 'walk_the_box_14.mp4', 'walk_the_box_28.mp4', 'walk_the_box_29.mp4', 'walk_the_box_15.mp4', 'walk_the_box_9.mp4', 'walk_the_box_8.mp4', 'walk_the_box_1.mp4', 'walk_the_box_3.mp4', 'walk_the_box_2.mp4', 'walk_the_box_6.mp4', 'walk_the_box_7.mp4', 'walk_the_box_5.mp4', 'walk_the_box_4.mp4', 'walk_the_box_18.mp4', 'walk_the_box_24.mp4', 'walk_the_box_30.mp4', 'walk_the_box_31.mp4', 'walk_the_box_25.mp4', 'walk_the_box_19.mp4', 'walk_the_box_33.mp4', 'walk_the_box_27.mp4', 'walk_the_box_26.mp4', 'walk_the_box_32.mp4', 'walk_the_box_22.mp4', 'walk_the_box_23.mp4', 'walk_the_box_21.mp4', 'walk_the_box_35.mp4', 'walk_the_box_34.mp4', 'walk_the_box_20.mp4']
['walk_the_box_11.mp4', 'walk_the_box_10.mp4', 'walk_the_box_12.mp4', 'walk_the_box_13.mp4', 'walk_the_box_17.mp4', 'walk_the_box_16.mp4', 'walk_the_box_14.mp4', 'walk_t

In [8]:
tagged_paths.items()

dict_items([('Walk The Box', [<tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_11.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_10.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_12.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_13.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_17.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_16.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_14.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_28.mp4'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_29.mp4'>

In [9]:
included_video_paths = []
labels = []
video_frames = [] 

total_videos = sum([len(v) for k, v in tagged_paths.items()])
with tqdm.tqdm(bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_noinv_fmt}]",
                desc="Saving videos...", unit=" videos", total=total_videos, leave=False) as pbar_full:
    label_count = len(tagged_paths)
    
    with tqdm.tqdm(bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_noinv_fmt}]", 
                    desc="Reading folders...", unit=" folders", total=label_count, leave=False) as pbar_folders:
        for category_index, (category, video_path_list) in enumerate(tagged_paths.items()):
            pbar_folders.set_description(f"Reading folder: \"{category}\"")
            
            video_count = len(video_path_list)
            with tqdm.tqdm(bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_noinv_fmt}]", 
                            desc="Reading videos...", unit= " videos", total=video_count, leave=False) as pbar_videos:
                for video_path in video_path_list:
                    decoded_video_path = video_path.numpy().decode("utf-8")
                    print(decoded_video_path)
                    dataset_tag, main_folder, category, video_name = decoded_video_path.split('/')
                    pbar_videos.set_description(f"Reading video: \"{video_name}\"")
                    print(decoded_video_path)
                    current_video_frames = read_video(decoded_video_path)
                    if len(current_video_frames) == dataset["sequence_length"]:
                        labels.append(category_index)
                        video_frames.append(current_video_frames)
                        included_video_paths.append(video_path)
                    
                    pbar_videos.update(1)
                    pbar_full.update(1)
                    
            pbar_folders.update(1)
                    
        
        print("Dataset read successfully.")
        

Reading folders...:   0%|          | 0/7 [00:00<?, ? folders/s]

dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_11.mp4
dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_11.mp4



















































Saving videos...:   0%|          | 1/245 [00:03<13:09,  0.31 videos/s]


dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_10.mp4
dataset/Fit3D Video Dataset/Walk The Box/walk_the_box_10.mp4



















































Saving videos...:   1%|          | 2/245 [00:06<13:06,  0.31 videos/s]


...

dataset/Fit3D Video Dataset/Burpees/burpees_35.mp4
dataset/Fit3D Video Dataset/Burpees/burpees_35.mp4















































Saving videos...: 100%|█████████▉| 244/245 [12:08<00:03,  0.32 videos/s]


dataset/Fit3D Video Dataset/Burpees/burpees_21.mp4
dataset/Fit3D Video Dataset/Burpees/burpees_21.mp4













































Saving videos...: 100%|██████████| 245/245 [12:11<00:00,  0.32 videos/s]
                                                                        

Dataset read successfully.


In [10]:

labeled_video_paths, videos = tagged_paths, video_frames         
videos = np.asarray(videos)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels, num_classes=len(labeled_video_paths))

X_train, X_test, Y_train, Y_test = train_test_split(videos, labels, test_size=0.2, shuffle=True)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)

X_train = np.asarray(X_train)
X_validation = np.asarray(X_validation)
X_test = np.asarray(X_test)
Y_train = np.asarray(Y_train)
Y_validation = np.asarray(Y_validation)
Y_test = np.asarray(Y_test)

**Training model**

In [11]:
from keras.models import Sequential
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
model = Sequential()
        
model.add(Conv3D(32,kernel_size=(3,3,3),activation='relu',input_shape=(75,128,128,1))),

model.add(MaxPooling3D(pool_size=(2,2,2))),

model.add(Conv3D(64,kernel_size=(3,3,3),activation='relu')),

model.add(MaxPooling3D(pool_size=(2,2,2))),

model.add(Conv3D(128,kernel_size=(3,3,3),activation='relu')),

model.add(MaxPooling3D(pool_size=(2,2,2))),

model.add(Conv3D(256,kernel_size=(3,3,3),activation='relu')),

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(128))

model.add(Dense(512,activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(7,activation='softmax'))

model.summary()

/Users/ataberkuygar/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 73, 126, 126,   │           896 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 36, 63, 63, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 34, 61, 61, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 17, 30, 30, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 15, 28, 28,     │       221,312 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 7, 14, 14, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 5, 12, 12, 256) │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 2, 6, 6, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 2, 9216)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │     4,784,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,016,839 (22.95 MB)

 Trainable params: 6,016,839 (22.95 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from keras.optimizers import Adam

In [16]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
training_history = model.fit(x=X_train,
                             y=Y_train,
                             epochs=5,
                             batch_size=8,
                             shuffle=True,
                             validation_data=(X_validation, Y_validation))


Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 225s 11s/step - accuracy: 0.9364 - loss: 0.2651 - val_accuracy: 0.8250 - val_loss: 0.4239
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 221s 11s/step - accuracy: 0.8314 - loss: 0.3937 - val_accuracy: 0.8750 - val_loss: 0.3704
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 223s 11s/step - accuracy: 0.8709 - loss: 0.2974 - val_accuracy: 0.8000 - val_loss: 0.3643
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 221s 11s/step - accuracy: 0.8896 - loss: 0.2802 - val_accuracy: 0.8500 - val_loss: 0.3293
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 221s 11s/step - accuracy: 0.8749 - loss: 0.2578 - val_accuracy: 0.8500 - val_loss: 0.3421


In [18]:
# Save the model
model.save("model.h5")
